## Building a small GPT decoder in order to generate La Fontaine's Fables

Inspired and learnt from Kartpathy's video on GPT :(https://karpathy.ai/zero-to-hero.html)

In [1]:
# read it in to inspect it
with open('laf.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  534444


In [3]:
# let's look at the first 1000 characters
print(text[:1000])

  Au lecteur

  Cette version électronique reproduit dans son intégralité
  la version originale.

  La ponctuation n'a pas été modifiée hormis quelques corrections
  mineures.

  L'orthographe a été conservée. Seuls quelques mots ont été modifiés.
  La liste des modifications se trouve à la fin du texte.




  FABLES

  DE

  LA FONTAINE


  [Illustration: Buste de La Fontaine]




  FABLES

  DE

  LA FONTAINE


  ILLUSTRATIONS

  PAR

  GRANDVILLE

  [Illustration]


  PARIS

  GARNIER FRÈRES, LIBRAIRES-ÉDITEURS

  6, RUE DES SAINTS-PÈRES--PALAIS-ROYAL, 215


  M DCCC LXVIII




[Illustration: A Monseigneur Le Dauphin]

MONSEIGNEUR,


S'IL y a quelque chose d'ingénieux dans la république des lettres,
on peut dire que c'est la manière dont Ésope a débité sa morale. Il
seroit véritablement à souhaiter que d'autres mains que les miennes y
eussent ajouté les ornements de la poésie, puisque le plus sage des
anciens a jugé qu'ils n'y étoient pas inutiles. J'ose, MONSEIGNEUR,
vous en prése

In [4]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !'()*,-.0123456789:;?ABCDEFGHIJLMNOPQRSTUVXYZ[]_abcdefghijklmnopqrstuvwxyz«»ÇÈÉÊÎàâæçèéêëîïôùûŒœήαβμπςτόϐἈ
108


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("Bonjour"))
print(decode(encode("Bonjour")))

[24, 64, 63, 59, 64, 70, 67]
Bonjour


In [7]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([534444]) torch.int64
tensor([ 1,  1, 23, 70,  1, 61, 54, 52, 69, 54, 70, 67,  0,  0,  1,  1, 25, 54,
        69, 69, 54,  1, 71, 54, 67, 68, 58, 64, 63,  1, 88, 61, 54, 52, 69, 67,
        64, 63, 58, 66, 70, 54,  1, 67, 54, 65, 67, 64, 53, 70, 58, 69,  1, 53,
        50, 63, 68,  1, 68, 64, 63,  1, 58, 63, 69, 88, 56, 67, 50, 61, 58, 69,
        88,  0,  1,  1, 61, 50,  1, 71, 54, 67, 68, 58, 64, 63,  1, 64, 67, 58,
        56, 58, 63, 50, 61, 54,  9,  0,  0,  1,  1, 33, 50,  1, 65, 64, 63, 52,
        69, 70, 50, 69, 58, 64, 63,  1, 63,  3, 50,  1, 65, 50, 68,  1, 88, 69,
        88,  1, 62, 64, 53, 58, 55, 58, 88, 54,  1, 57, 64, 67, 62, 58, 68,  1,
        66, 70, 54, 61, 66, 70, 54, 68,  1, 52, 64, 67, 67, 54, 52, 69, 58, 64,
        63, 68,  0,  1,  1, 62, 58, 63, 54, 70, 67, 54, 68,  9,  0,  0,  1,  1,
        33,  3, 64, 67, 69, 57, 64, 56, 67, 50, 65, 57, 54,  1, 50,  1, 88, 69,
        88,  1, 52, 64, 63, 68, 54, 67, 71, 88, 54,  9,  1, 40, 54, 70, 61, 68,
       

In [8]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
train_data[:block_size+1]

tensor([ 1,  1, 23, 70,  1, 61, 54, 52, 69])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([1]) the target: 1
when input is tensor([1, 1]) the target: 23
when input is tensor([ 1,  1, 23]) the target: 70
when input is tensor([ 1,  1, 23, 70]) the target: 1
when input is tensor([ 1,  1, 23, 70,  1]) the target: 61
when input is tensor([ 1,  1, 23, 70,  1, 61]) the target: 54
when input is tensor([ 1,  1, 23, 70,  1, 61, 54]) the target: 52
when input is tensor([ 1,  1, 23, 70,  1, 61, 54, 52]) the target: 69


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[ 1,  1,  1,  1, 27, 69,  1, 71],
        [ 1, 68, 54,  1, 71, 54, 67, 67],
        [63, 69,  1, 54, 69,  1, 62, 50],
        [52, 64, 70, 22,  0,  1,  1,  1]])
targets:
torch.Size([4, 8])
tensor([[ 1,  1,  1, 27, 69,  1, 71, 54],
        [68, 54,  1, 71, 54, 67, 67, 50],
        [69,  1, 54, 69,  1, 62, 50, 58],
        [64, 70, 22,  0,  1,  1,  1,  1]])
----
when input is [1] the target: 1
when input is [1, 1] the target: 1
when input is [1, 1, 1] the target: 1
when input is [1, 1, 1, 1] the target: 27
when input is [1, 1, 1, 1, 27] the target: 69
when input is [1, 1, 1, 1, 27, 69] the target: 1
when input is [1, 1, 1, 1, 27, 69, 1] the target: 71
when input is [1, 1, 1, 1, 27, 69, 1, 71] the target: 54
when input is [1] the target: 68
when input is [1, 68] the target: 54
when input is [1, 68, 54] the target: 1
when input is [1, 68, 54, 1] the target: 71
when input is [1, 68, 54, 1, 71] the target: 54
when input is [1, 68, 54, 1, 71, 54] the target:

In [12]:
print(xb) # our input to the transformer

tensor([[ 1,  1,  1,  1, 27, 69,  1, 71],
        [ 1, 68, 54,  1, 71, 54, 67, 67],
        [63, 69,  1, 54, 69,  1, 62, 50],
        [52, 64, 70, 22,  0,  1,  1,  1]])


In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparamètres
batch_size = 64  # combien de séquences indépendantes allons-nous traiter en parallèle ?
block_size = 128  # quelle est la longueur de contexte maximale pour les prédictions ?
max_iters = 3000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' si torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 200
n_head = 5
n_layer = 4
dropout = 0.2
# ------------

torch.manual_seed(1337)

# Chargement du texte
with open('laf.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Liste des caractères uniques dans le texte
chars = sorted(list(set(text)))
vocab_size = len(chars)
# Création d'un mapping des caractères vers des entiers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encodeur : prend une chaîne, renvoie une liste d'entiers
decode = lambda l: ''.join([itos[i] for i in l]) # décodeur : prend une liste d'entiers, renvoie une chaîne

# Division des données en ensemble d'entraînement et de validation
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data)) # 90% pour l'entraînement, le reste pour la validation
train_data = data[:n]
val_data = data[n:]

# Chargement des lots de données
def get_batch(split):
    # génère un petit lot de données avec des entrées x et des cibles y
    data = train_data si split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ une tête de self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # Calcul des scores d'attention ("affinités") en utilisant torch.einsum
        wei = torch.einsum('btc,bsc->bts', q, k) * C**-0.5 # (B, T, T)  # NORMALISATIOn par sqrt(C)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
         # ou sinon:
        # mask = torch.arange(T)[None, :] > torch.arange(T)[:, None]
        # wei = wei.masked_fill(mask, float('-inf'))
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # Agrégation pondérée des valeurs
        v = self.value(x) # (B,T,C)
        out = torch.einsum('bts,bsc->btc', wei, v) # (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ plusieurs têtes de self-attention en parallèle """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ une simple couche linéaire suivie d'une non-linéarité """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Bloc Transformer : communication suivie de calculs """

    def __init__(self, n_embd, n_head):
        # n_embd : dimension des embeddings, n_head : nombre de têtes
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    """ Modèle de langage basé sur les bigrammes """

    def __init__(self):
        super().__init__()
        # chaque token lit directement les logits pour le token suivant à partir d'une table de lookup
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # normalisation finale
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx et targets sont tous deux des tenseurs (B,T) d'entiers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens, temperature=1.0):
        # idx est un tableau (B, T) des indices dans le contexte actuel
        for _ in range(max_new_tokens):
            # couper idx aux derniers block_size tokens
            idx_cond = idx[:, -block_size:]
            # obtenir les prédictions
            logits, loss = self(idx_cond) #(B,taille_phrase) vers (B,taille_phrase, vocab_size)
            # ne se concentrer que sur la dernière étape temporelle
            logits = logits[:, -1, :] / temperature # devient (B, C)
            #PLUTOT QUE [:][-1][:]:  cela sélectionne le dernier élément du batch entier, et non pas le dernier token pour chaque élément du batch.
            # appliquer softmax pour obtenir les probabilités
            probs = F.softmax(logits, dim=-1) # (B, C)
            # échantillonner à partir de la distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # ajouter l'indice échantillonné à la séquence en cours
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# imprimer le nombre de paramètres du modèle
print(sum(p.numel() for p in m.parameters())/1e6, 'M paramètres')

# créer un optimiseur PyTorch
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # de temps en temps, évaluer la perte sur les ensembles train et val
    si iter % eval_interval == 0 ou iter == max_iters - 1:
        losses = estimate_loss()
        print(f"étape {iter}: perte d'entraînement {losses['train']:.4f}, perte de validation {losses['val']:.4f}")

    # échantillonner un lot de données
    xb, yb = get_batch('train')

    # évaluer la perte
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# générer du texte à partir du modèle
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000, temperature=0.8)[0].tolist()))


1.997308 M parameters
step 0: train loss 4.7935, val loss 4.7837
step 500: train loss 1.6892, val loss 1.8160
step 1000: train loss 1.4358, val loss 1.6734
step 1500: train loss 1.3163, val loss 1.5428
step 2000: train loss 1.2375, val loss 1.4166
step 2500: train loss 1.1883, val loss 1.4123
step 2999: train loss 1.1468, val loss 1.4251

  [14] ANINIB.


  [11] Entre assec: d'assa part gracer.

  [Illustration]




  [Illustration]

  XVIII

  LA BOURGENME.


  Est-il moins quelque saurieux,
  Comment sens fit troute en le peau mordant,
  Ulysses,
        Souvent sur son tombe: et sauvel impilage:
  Vous m'assissent anima6Mes leur œur, sous de plus instruiens,
          Mais friers sur tous les accendentes:
          Et mot sort vos autant sort été,
          Qu'un peuple se veur maison,
          En admestènelle un duisan ma visoit.

  [Illustration]





  [Illustration]

  XVII

  LE PELLE ET LE FILHERVÉ.


               Reposopit un enfant du phiebis.

            La promemière à

In [2]:
for iter in range(1000):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

step 0: train loss 1.1432, val loss 1.4321
step 500: train loss 1.1127, val loss 1.3801





  [Illustration]

  VII

  L'ÉPHÉQUISSEUR.


              Je mets essuice au mort,
          Quelque ne croira pas menu;
  Si des toiux que nous un bien part: cependant
  Que n'est bâtir, ni qui galands qu'on monte.
          Quand il est rat son éluté mangé,
            Tous les champs d'affare Nécepter;
  Je ponse à d'apporté, ôtant au vrai, veuger
 ,        Entre magistrabler, pillis des babors.
  Je parle à moi dépense
  A dit inventure; sa coquisable de ne plaisant alard,
  Qu'à soi donnant à son moissant que je part trop beaucoupe
  Tous aussi. Moi, je le blâme. Comme Prit à la vie,
          Amour traînt de natirer,
  Riant, gule-malle, et croyé d'exégère,
          Qui se pand plus surpas, à moi,
  Demeurer jeune le temps t'aille pays, une belle,
  Qui, ferasse-pologuede? Xantus,
  Ainsi, ne n'y parla va point aussitôt
          Se seroit tous regassent et davantagré.
          Le libr

In [3]:
for iter in range(1000):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

step 0: train loss 1.0855, val loss 1.3901
step 500: train loss 1.0622, val loss 1.3971

succès d'un inspniteur
          Passent-ils plairessant avoir plus de retemin
  Esclavir d'autre difficul. Il fut bien
  Que fournit pour tant si chacun s'embrassa d'un crimatière;
  Sur quand la fille faitre, et l'objet, elle mit à l'or.
  Nous délibarre ainsi donce, si philosophème.
  [Illustration]




  [Illustration]

  IV

  LE PAROSSEUR.



  n'étoit pas le coloup, de grandeur au gland:
  Commez dit de la diluttre,
  Et l'autre besoin, se plus-on son jour
          Comme il pabusoit un testant,
          Je le Jadis, qu'on peut chercher, pour  contre cet objet:
  Tout au soin dur le vieillar faire à la Pinteur
          Aillez viter. Ainsi pas craint:
  Il ne me mette tout les âmes un auteure en seul?
  Que m'affrère oblige avec trio que père:
  Le bois, qui ne croit qu'on l'en avoit tombé:
          Prend je récidissant pesé?
  C'est là besoin, sitôt Plup-Repors, de façon repressenti,
  De